<a href="https://colab.research.google.com/github/arkalim/PyTorch/blob/master/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resnet Architecture
For official model Code

https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py


![alt text](https://cdn-images-1.medium.com/max/1600/1*aq0q7gCvuNUqnMHh4cpnIw.png)

In [0]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torchvision import models

## Pretrained weights

In [0]:
from torch.hub import load_state_dict_from_url

__all__ = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

## Residual Blocks![alt text](https://cdn-images-1.medium.com/max/1600/1*j_lC2gsO1Kbia8PIQGHUZg.png)

##Implementing Basic Block


In [0]:
class BasicBlock(nn.Module):
    '''Basic Block is used in resnet 18 and 34. It consists of two 3x3 convolutions.
       Relu activation function is added after adding the skip connection. There is no 
       expansion of features in case of basic blocks'''
    expansion = 1
    def __init__(self , input_channels , output_channels , stride = 1 , dim_change = None):
        super(BasicBlock , self).__init__()
        self.conv1 = nn.Conv2d(input_channels , output_channels , stride = stride , kernel_size = 3 , padding = 1, bias=False)
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels , output_channels , stride = 1 , kernel_size = 3 , padding = 1, bias=False)
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.dim_change = dim_change
        
    def forward(self , x):
        residual_connection = x
        output = F.relu(self.bn1(self.conv1(x)))
        # relu to be added after concatenation
        output = self.bn2(self.conv2(output))
        
        #But first we will check for dimension change
        if self.dim_change is not None:
            residual_connection = self.dim_change(residual_connection)
            
        output = residual_connection + output
        output = F.relu(output)
        return output

###Implementing Bottleneck block

In [0]:
class BottleNeck(nn.Module):
    '''Bottleneck blocks are used in Resnet 50, 101, and 152. A general bottleneck block consists of a 1x1
       conv followed by a 3x3 conv and another 1x1 conv. The first 1x1 conv reduces the number of filters 
       to speedup feature extraction by 3x3 conv. The last 1x1 conv increases the number of filters back
       to that of the input before adding the skip connection.'''
    
    expansion = 4
    def __init__(self , input_channels , output_channels , stride = 1 , downsample = None):
        super(BottleNeck , self).__init__()
        self.conv1 = nn.Conv2d(input_channels , output_channels , stride = 1 , kernel_size = 1, bias=False)
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels , output_channels , stride = stride , padding = 1 , kernel_size = 3, bias=False)
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.conv3 = nn.Conv2d(output_channels , output_channels*4 , stride = 1 , kernel_size = 1, bias=False)
        self.bn3 = nn.BatchNorm2d(output_channels*4)
        self.downsample = downsample
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self , x):
        residual_connection = x
        output = F.relu(self.bn1(self.conv1(x)))
        output = F.relu(self.bn2(self.conv2(output)))
        output = self.bn3(self.conv3(output))
        
        if self.downsample is not None:
            residual_connection = self.downsample(residual_connection)
            
        output = output + residual_connection
        output = self.relu(output)
        return output
    

## Resnet Class

In [0]:
class ResNet(nn.Module):
    def __init__(self , block , num_layers , classes = 1000):
        '''
        The block can be bottleneck block or basic block and num_layers is the
        number of layers required for each block. num_layers will be a list containing 
        the number of blocks in each layer
        '''
        super(ResNet , self).__init__()
        self.input_size = 64
        
        # First layer in all is 7*7 convolution with stride 2, it contains 64 channels
        self.conv1 = nn.Conv2d(3, 64 , stride = 2 , kernel_size = 7, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # comment maxpool if you want to use it for cifar10 since image size is 
        # small 32*32 and maxpool would make it even smaller for processing
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        ##################### Main Building Blocks ####################
        
        # for 1st layer(block) all the 3x3 convolutions will take place using 64 filters
        # for 2nd block, 128 filters and so on.
        self.layer1 = self.make_layer(block , 64 , num_layers[0] , stride = 1)
        self.layer2 = self.make_layer(block , 128 , num_layers[1] , stride = 2)
        self.layer3 = self.make_layer(block , 256 , num_layers[2] , stride = 2)
        self.layer4 = self.make_layer(block , 512 , num_layers[3] , stride = 2)
        
        # last linear layer consisting of 2048 filters as input and num_classes as output
        self.fc = nn.Linear(512*block.expansion , classes)
        
    def make_layer(self , block , input_size ,num_layers , stride=1):
        downsample = None
        # we have to find the condition for change of dimensions
        # basically saying that change for dimensions 128,256,512
        # input size gets updated later
        if stride!=1 or input_size!=self.input_size*block.expansion:
            downsample = nn.Sequential(nn.Conv2d(self.input_size , input_size*block.expansion , kernel_size = 1 , 
                                                 stride = stride, bias=False),
                                       nn.BatchNorm2d(input_size*block.expansion))
            
        # empty list to append the layers to    
        layers = []
        # appending the layers
        layers.append(block(self.input_size , input_size , stride , downsample))
        self.input_size = input_size*block.expansion
        
        for i in range(1,num_layers):
            layers.append(block(self.input_size , input_size))
            
        return nn.Sequential(*layers)
        
    def forward(self , x):
        # initial convolution 
        x = F.relu(self.maxpool(self.bn1(self.conv1(x))))
        
        # The 4 layers (blocks)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = F.avg_pool2d(x, kernel_size=4) 
        x = x.reshape(x.size(0) , -1)
        x = self.fc(x)
        return x          

## Create Resnets

In [6]:
def Create_Resnet(arch='resnet50', pretrained = False, classes=1000):
    
    if arch == 'resnet18':
        resnet = ResNet(BasicBlock , [2,2,2,2] , classes)
    elif arch == 'resnet34':
        resnet = ResNet(BasicBlock , [3,4,6,3] , classes)
    elif arch == 'resnet50':
        resnet = ResNet(BottleNeck , [3,4,6,3] , classes)
    elif arch == 'resnet101':
        resnet = ResNet(BottleNeck , [3,4,23,3] , classes)
    elif arch == '152':
        resnet = ResNet(BottleNeck , [3,8,36,3] , classes)

    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],progress=True)
        resnet.load_state_dict(state_dict)
    return resnet

resnet = Create_Resnet('resnet50', True)

print(resnet)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:03<00:00, 30439409.96it/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BottleNeck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=

##Visualizing the network using ONNX

In [7]:
!pip install onnx
sample_input = torch.randn(1, 3, 224, 224)

filename = "resnet50.onnx"
torch.onnx.export(resnet, sample_input, filename)
print("Network Exported as: {}".format(filename))

     |████████████████████████████████| 7.1MB 29.8MB/s 
Network Exported as: resnet50.onnx


Now download the onnx file and run it in Netron